In [1]:
!pip install python-dotenv pyyaml langchain langchain-experimental

In [2]:
import os
from dotenv import load_dotenv
import yaml
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain.sql_database import SQLDatabase
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX, _postgres_prompt
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.llms import Bedrock
from langchain.prompts.example_selector.semantic_similarity import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain_experimental.sql import SQLDatabaseChain
from env import *

In [3]:
#load env variables, setup llm instance profile and snowflake URI

load_dotenv()

llm = Bedrock(
    credentials_profile_name=os.getenv("snowadmin"),
    model_id="anthropic.claude-v1",
    endpoint_url="https://bedrock-runtime.us-east-1.amazonaws.com",
    region_name="us-east-1",
    verbose=True
)

def get_snowflake_uri():
    snowflake_url = f"snowflake://{user}:{password}@{snowflake_account}/{database}/{schema}?warehouse={warehouse_name}&role={role}"
    return snowflake_url

In [4]:
#Load the sql examples for few-shot prompting examples
def load_samples():
    sql_samples = None
    with open("netflix_examples.yaml", "r") as stream:
        sql_samples = yaml.safe_load(stream)
    return sql_samples

In [5]:
# This is formatting the prompts that are retrieved from the sample prompy yaml and loads the examples, 
# creates the embeddings, stores them in vector db, and it returns the 3 most similar

def load_few_shot_chain(llm, db, examples):
    example_prompt = PromptTemplate(
        input_variables=["table_info", "input", "sql_cmd", "sql_result", "answer"],
        template=(
            "{table_info}\n\nQuestion: {input}\nSQLQuery: {sql_cmd}\nSQLResult:"
            " {sql_result}\nAnswer: {answer}"
        ),
    )
    local_embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    example_selector = SemanticSimilarityExampleSelector.from_examples(
        examples,
        local_embeddings,
        Chroma,
        k=min(3, len(examples)),
    )
    few_shot_prompt = FewShotPromptTemplate(
        example_selector=example_selector,
        example_prompt=example_prompt,
        prefix=_postgres_prompt + "Provide no preamble" + " Here are some examples:",
        suffix=PROMPT_SUFFIX,
        input_variables=["table_info", "input", "top_k"],
    )
    return SQLDatabaseChain.from_llm(
        llm,
        db,
        prompt=few_shot_prompt,
        use_query_checker=True,
        verbose=True,
        return_intermediate_steps=True,
    )


In [6]:
# Executing the SQL query on snowflake
def snowflake_answer(question):
    snowflake_url = get_snowflake_uri()
    db = SQLDatabase.from_uri(snowflake_url, sample_rows_in_table_info=1, include_tables=["data"])
    examples = load_samples()
    sql_db_chain = load_few_shot_chain(llm, db, examples)
    answer = sql_db_chain(question)

In [7]:
# Example usage
if __name__ == "__main__":
    # Sample question
    sample_question = "List all titles with TYPE show and have SEASON lesser than 4 and contains GENRE comedy?"

    # Call the snowflake_answer function with the sample question
    snowflake_answer(sample_question)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/snowflake/connector/options.py:103: UserWarning: You have an incompatible version of 'pyarrow' installed (14.0.1), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(




> Entering new SQLDatabaseChain chain...
List all titles with TYPE show and have SEASON lesser than 4 and contains GENRE comedy?
SQLQuery:SELECT title FROM data WHERE type = 'SHOW' AND seasons < 4 AND genres LIKE '%comedy%'
SQLResult: [('Monty Pythons Fliegender Zirkus',), ('High Risk',), ('Cowboy Bebop',), ('Sonic X',), ('Chappelles Show',), ('Gurren Lagann',), ('Black Butler',), ('Boys Over Flowers',), ('Dead Set',), ('Toradora!',), ('Angel Beats!',), ('Monty Python: Almost the Truth (The Lawyers Cut)',), ('Ouran High School Host Club',), ('Iris',), ('DreamWorks Shreks Swamp Stories',), ('Ben & Hollys Little Kingdom',), ('Den-noh Coil',), ('Zig and Sharko',), ('The Cartel',), ('Kung Fu Panda Awesome Secrets',), ('Dreamworks Happy Holidays from Madagascar',), ('Timmy Time',), ('Oscars Oasis',), ('Fated to Love You',), ('Nuevo Rico Nuevo Pobre',), ('Pecados capitales',), ('Pedro El Escamoso',), ('Monty Pythons Personal Best',), ('Together',), ('Black & White',), ('The Prince Who Turn